In [2]:
### Happy XMAS for Christians, Happy New Year for Christians and Christians. from hklee

rudolf_lyrics = """
You know Dasher, and Dancer, and Prancer, and Vixen
Comet, and Cupid, and Donner and Blitzen
But do you recall the most famous reindeer of all

Rudolph the red-nosed reindeer had a very shiny nose
And if you ever saw it, you would even say it glows
All of the other reindeer used to laugh and call him names
They never let poor Rudolph join in any reindeer games

Then one foggy Christmas Eve Santa came to say
"Rudolph with your nose so bright, won't you guide my sleigh tonight?"
Then how the reindeer loved him as they shouted out with glee
"Rudolph the red-nosed reindeer, you'll go down in history"

Rudolph the red-nosed reindeer had a very shiny nose
And if you ever saw it, you would even say it glows
All of the other reindeer used to laugh and call him names
They never let poor Rudolph join in any reindeer games

Then one foggy Christmas Eve Santa came to say
"Rudolph with your nose so bright, won't you guide my sleigh tonight?"
Then how the reindeer loved him as they shouted out with glee
"Rudolph the red-nosed reindeer, you'll go down in history"
""".replace('\n', '*')


import numpy as np, pandas as pd

print('load data')
child_prefs = pd.read_csv('input/child_wishlist.csv', header=None).values[:,1:]
gift_prefs = pd.read_csv('input/gift_goodkids.csv', header=None).values[:,1:]

n_children = 1000000 # n children to give
n_gift_type = 1000 # n types of gifts available
n_gift_quantity = 1000 # each type of gifts are limited to this quantity
n_child_pref = 10 # number of gifts a child ranks
n_gift_pref = 1000 # number of children a gift ranks
twins = 4000
ratio_gift_happiness = 2
ratio_child_happiness = 2

print('combine all nontrivial child-gift pairs and assign happiness on them.')
df_santa = pd.DataFrame({
    'gift':np.repeat(np.arange(n_gift_type), n_gift_pref) ,
    'child':gift_prefs.flatten(),
    'santa_happ':np.tile(np.linspace(n_gift_pref, 1, n_gift_pref) * 2, (n_gift_type, 1)).flatten()})

df_child = pd.DataFrame({
    'child':np.repeat(np.arange(n_children), n_child_pref),
    'gift': child_prefs.flatten(),
    'child_happ': np.tile(np.linspace(n_child_pref, 1, n_child_pref) * 2, (n_children, 1)).flatten()})

df = df_child.merge(df_santa, on=['child', 'gift'], how='outer')

df['child_happ'] = df.child_happ.fillna(-1)
df['santa_happ'] = df.santa_happ.fillna(-1)

max_child_happ = 2 * n_child_pref
max_gift_happ  = 2 * n_gift_pref

df['happiness']  = df.child_happ / max_child_happ + df.santa_happ / max_gift_happ    

print('split singles and twins, and take care of twins')
df_twins = df[df.child < twins].reset_index(drop=True)
df_twins['child_t'] = df_twins.child.apply(lambda x: x + 1 if x//2*2==x else x - 1)  # other me

# add happiness of the other's choice
df_tmp = df_twins[['child_t', 'gift']].merge(df_twins[['child', 'gift', 'child_happ']], left_on=['child_t', 'gift'], right_on=['child', 'gift'], how='left')
df_twins['child_happ_t'] = df_tmp.child_happ.fillna(-1)

df_tmp = df_twins[['child_t', 'gift']].merge(df_santa, left_on=['child_t', 'gift'], right_on=['child', 'gift'], how='left')
df_twins['santa_happ_t'] = df_tmp.santa_happ.fillna(-1)

df_twins['happiness'] = (df_twins.child_happ + df_twins.child_happ_t) / max_child_happ + (df_twins.santa_happ + df_twins.santa_happ_t) / max_gift_happ 

# remove other me 
ii = df_twins.child > df_twins.child_t
df_twins.loc[ii, 'child'] = df_twins.child_t[ii]

df_twins = df_twins[['child', 'gift', 'happiness']].copy()
df_twins['n'] = 2

print('merge singles and twins')
df1 = df[['child', 'gift', 'happiness']][df.child >= twins].reset_index(drop=True)
df1['n'] = 1

df_dist = pd.concat([df_twins, df1], ignore_index=True)
df_dist.sort_values(by='happiness', ascending=False, inplace=True)
df_dist.reset_index(drop=True, inplace=True)

print('create lists of candidate children and their happiness on each gift. it takes some secs.')
candidates = []  # for gifts
chappiness = []
for gift in range(n_gift_type):
    a = df_dist[df_dist.gift==gift][['child', 'happiness']]
    candidates.append(a.child.values)
    chappiness.append(a.happiness.values)
    
clen = np.array([len(x) for x in candidates])
crt = np.zeros_like(clen)    

print('distribute gifts by filling gift list uniformly.')
preds = np.ones(n_children, dtype=np.int32) * -1
lefts = np.ones(n_gift_type, dtype=np.int32) * 1000

total_happ = 0

lefts_blocked = np.zeros_like(lefts)

for i in range(n_children):
    
    gifts = np.argsort(lefts)[::-1]
    
    maxlefts = lefts[gifts[0]]
    if maxlefts == 0: break
    
    maxhapp = -1000
    maxchild = -1
    maxgift  = -1
    
    for gift in gifts:
        if lefts[gift] != maxlefts:
            break
            
        candid = candidates[gift]
        chapp  = chappiness[gift]
        k = crt[gift]

        while (k < clen[gift]):
            child = candid[k]
            if preds[child] == -1: # not assigned yet
                happ  = chapp[k]
                if (happ > maxhapp) and ( (child >= twins) or (lefts[gift] >= 2)):
                    maxhapp  = happ
                    maxchild = child
                    maxgift  = gift
                break
            else:
                k = k + 1
        crt[gift] = k
        
    if maxchild == -1: 
        for gift in gifts:
            if lefts[gift] != maxlefts:
                break
            lefts_blocked[gift] = lefts[gift]
            lefts[gift] = 0
    else:
        lefts[maxgift] -= 1
        preds[maxchild] = maxgift
        if maxchild < twins:
            lefts[maxgift] -= 1
            preds[maxchild+1] = maxgift
        crt[maxgift] += 1
        total_happ += maxhapp
        
    if i % 500 == 0:
        print("{}/{}, happiness={:.4f} {}".format(i, n_children, total_happ / 1000000, rudolf_lyrics[(i//500)%len(rudolf_lyrics)]))
        
#expected score
#expected_happiness = (total_happ + np.sum(lefts_blocked) * (-1/max_child_happ + -1/max_gift_happ)) / ((n_children + n_gift_type * n_gift_quantity)/2)
expected_happiness = (total_happ + np.sum(lefts_blocked) * (-1/max_child_happ + -1/max_gift_happ)) / 1000000
print('expected happiness={:.4f}'.format(expected_happiness))

# give remaining gifts to unfortunate kids.
lefts = lefts_blocked

gifts = []
for gift in np.where(lefts >= 2)[0]:
    n = lefts[gift] // 2 * 2  # twins comes first
    lefts[gift] -= n
    gifts = gifts + [ gift ] * n

for gift in np.where(lefts ==1)[0]:
    gifts.append(gift)
    
preds[preds == -1] = np.array(gifts)

# submit
pd.DataFrame({'ChildId':np.arange(n_children), 'GiftId':preds}).set_index('ChildId').to_csv('uniform_fill.csv')

print('done.')

load data
combine all nontrivial child-gift pairs and assign happiness on them.
split singles and twins, and take care of twins
merge singles and twins
create lists of candidate children and their happiness on each gift. it takes some secs.
distribute gifts by filling gift list uniformly.
0/1000000, happiness=0.0000 *
500/1000000, happiness=0.0009 Y
1000/1000000, happiness=0.0016 o
1500/1000000, happiness=0.0024 u
2000/1000000, happiness=0.0030  
2500/1000000, happiness=0.0037 k
3000/1000000, happiness=0.0043 n
3500/1000000, happiness=0.0050 o
4000/1000000, happiness=0.0055 w
4500/1000000, happiness=0.0061  
5000/1000000, happiness=0.0067 D
5500/1000000, happiness=0.0072 a
6000/1000000, happiness=0.0078 s
6500/1000000, happiness=0.0083 h
7000/1000000, happiness=0.0088 e
7500/1000000, happiness=0.0094 r
8000/1000000, happiness=0.0099 ,
8500/1000000, happiness=0.0104  
9000/1000000, happiness=0.0109 a
9500/1000000, happiness=0.0114 n
10000/1000000, happiness=0.0119 d
10500/1000000, happi

116500/1000000, happiness=0.1181 v
117000/1000000, happiness=0.1186 e
117500/1000000, happiness=0.1191 n
118000/1000000, happiness=0.1196  
118500/1000000, happiness=0.1201 s
119000/1000000, happiness=0.1206 a
119500/1000000, happiness=0.1211 y
120000/1000000, happiness=0.1215  
120500/1000000, happiness=0.1220 i
121000/1000000, happiness=0.1225 t
121500/1000000, happiness=0.1230  
122000/1000000, happiness=0.1235 g
122500/1000000, happiness=0.1240 l
123000/1000000, happiness=0.1245 o
123500/1000000, happiness=0.1250 w
124000/1000000, happiness=0.1255 s
124500/1000000, happiness=0.1260 *
125000/1000000, happiness=0.1265 A
125500/1000000, happiness=0.1270 l
126000/1000000, happiness=0.1275 l
126500/1000000, happiness=0.1280  
127000/1000000, happiness=0.1285 o
127500/1000000, happiness=0.1290 f
128000/1000000, happiness=0.1295  
128500/1000000, happiness=0.1300 t
129000/1000000, happiness=0.1305 h
129500/1000000, happiness=0.1310 e
130000/1000000, happiness=0.1315  
130500/1000000, happ

234000/1000000, happiness=0.2341 e
234500/1000000, happiness=0.2346 i
235000/1000000, happiness=0.2351 g
235500/1000000, happiness=0.2356 h
236000/1000000, happiness=0.2360  
236500/1000000, happiness=0.2365 t
237000/1000000, happiness=0.2370 o
237500/1000000, happiness=0.2375 n
238000/1000000, happiness=0.2380 i
238500/1000000, happiness=0.2385 g
239000/1000000, happiness=0.2390 h
239500/1000000, happiness=0.2395 t
240000/1000000, happiness=0.2400 ?
240500/1000000, happiness=0.2405 "
241000/1000000, happiness=0.2409 *
241500/1000000, happiness=0.2414 T
242000/1000000, happiness=0.2419 h
242500/1000000, happiness=0.2424 e
243000/1000000, happiness=0.2429 n
243500/1000000, happiness=0.2434  
244000/1000000, happiness=0.2439 h
244500/1000000, happiness=0.2444 o
245000/1000000, happiness=0.2449 w
245500/1000000, happiness=0.2454  
246000/1000000, happiness=0.2458 t
246500/1000000, happiness=0.2463 h
247000/1000000, happiness=0.2468 e
247500/1000000, happiness=0.2473  
248000/1000000, happ

351500/1000000, happiness=0.3486 t
352000/1000000, happiness=0.3491  
352500/1000000, happiness=0.3496 g
353000/1000000, happiness=0.3500 l
353500/1000000, happiness=0.3505 o
354000/1000000, happiness=0.3510 w
354500/1000000, happiness=0.3515 s
355000/1000000, happiness=0.3520 *
355500/1000000, happiness=0.3525 A
356000/1000000, happiness=0.3529 l
356500/1000000, happiness=0.3534 l
357000/1000000, happiness=0.3539  
357500/1000000, happiness=0.3544 o
358000/1000000, happiness=0.3549 f
358500/1000000, happiness=0.3554  
359000/1000000, happiness=0.3558 t
359500/1000000, happiness=0.3563 h
360000/1000000, happiness=0.3568 e
360500/1000000, happiness=0.3573  
361000/1000000, happiness=0.3578 o
361500/1000000, happiness=0.3582 t
362000/1000000, happiness=0.3587 h
362500/1000000, happiness=0.3592 e
363000/1000000, happiness=0.3597 r
363500/1000000, happiness=0.3602  
364000/1000000, happiness=0.3606 r
364500/1000000, happiness=0.3611 e
365000/1000000, happiness=0.3616 i
365500/1000000, happ

469000/1000000, happiness=0.4610 g
469500/1000000, happiness=0.4615 h
470000/1000000, happiness=0.4620 t
470500/1000000, happiness=0.4624 ?
471000/1000000, happiness=0.4629 "
471500/1000000, happiness=0.4634 *
472000/1000000, happiness=0.4639 T
472500/1000000, happiness=0.4643 h
473000/1000000, happiness=0.4648 e
473500/1000000, happiness=0.4652 n
474000/1000000, happiness=0.4657  
474500/1000000, happiness=0.4662 h
475000/1000000, happiness=0.4667 o
475500/1000000, happiness=0.4671 w
476000/1000000, happiness=0.4676  
476500/1000000, happiness=0.4681 t
477000/1000000, happiness=0.4686 h
477500/1000000, happiness=0.4690 e
478000/1000000, happiness=0.4695  
478500/1000000, happiness=0.4700 r
479000/1000000, happiness=0.4705 e
479500/1000000, happiness=0.4709 i
480000/1000000, happiness=0.4714 n
480500/1000000, happiness=0.4719 d
481000/1000000, happiness=0.4724 e
481500/1000000, happiness=0.4728 e
482000/1000000, happiness=0.4733 r
482500/1000000, happiness=0.4737  
483000/1000000, happ

586500/1000000, happiness=0.5706 c
587000/1000000, happiness=0.5711 a
587500/1000000, happiness=0.5715 l
588000/1000000, happiness=0.5720 l
588500/1000000, happiness=0.5724  
589000/1000000, happiness=0.5729 t
589500/1000000, happiness=0.5733 h
590000/1000000, happiness=0.5738 e
590500/1000000, happiness=0.5743  
591000/1000000, happiness=0.5748 m
591500/1000000, happiness=0.5752 o
592000/1000000, happiness=0.5757 s
592500/1000000, happiness=0.5761 t
593000/1000000, happiness=0.5766  
593500/1000000, happiness=0.5770 f
594000/1000000, happiness=0.5775 a
594500/1000000, happiness=0.5779 m
595000/1000000, happiness=0.5784 o
595500/1000000, happiness=0.5788 u
596000/1000000, happiness=0.5793 s
596500/1000000, happiness=0.5798  
597000/1000000, happiness=0.5802 r
597500/1000000, happiness=0.5807 e
598000/1000000, happiness=0.5812 i
598500/1000000, happiness=0.5816 n
599000/1000000, happiness=0.5821 d
599500/1000000, happiness=0.5825 e
600000/1000000, happiness=0.5830 e
600500/1000000, happ

704000/1000000, happiness=0.6767 i
704500/1000000, happiness=0.6771 n
705000/1000000, happiness=0.6776  
705500/1000000, happiness=0.6780 a
706000/1000000, happiness=0.6785 n
706500/1000000, happiness=0.6789 y
707000/1000000, happiness=0.6794  
707500/1000000, happiness=0.6798 r
708000/1000000, happiness=0.6803 e
708500/1000000, happiness=0.6807 i
709000/1000000, happiness=0.6812 n
709500/1000000, happiness=0.6816 d
710000/1000000, happiness=0.6821 e
710500/1000000, happiness=0.6825 e
711000/1000000, happiness=0.6830 r
711500/1000000, happiness=0.6834  
712000/1000000, happiness=0.6839 g
712500/1000000, happiness=0.6843 a
713000/1000000, happiness=0.6848 m
713500/1000000, happiness=0.6852 e
714000/1000000, happiness=0.6857 s
714500/1000000, happiness=0.6861 *
715000/1000000, happiness=0.6866 *
715500/1000000, happiness=0.6870 T
716000/1000000, happiness=0.6875 h
716500/1000000, happiness=0.6879 e
717000/1000000, happiness=0.6883 n
717500/1000000, happiness=0.6888  
718000/1000000, happ

821500/1000000, happiness=0.7800  
822000/1000000, happiness=0.7804 y
822500/1000000, happiness=0.7809 o
823000/1000000, happiness=0.7812 u
823500/1000000, happiness=0.7817 '
824000/1000000, happiness=0.7821 l
824500/1000000, happiness=0.7826 l
825000/1000000, happiness=0.7830  
825500/1000000, happiness=0.7835 g
826000/1000000, happiness=0.7838 o
826500/1000000, happiness=0.7843  
827000/1000000, happiness=0.7847 d
827500/1000000, happiness=0.7852 o
828000/1000000, happiness=0.7856 w
828500/1000000, happiness=0.7861 n
829000/1000000, happiness=0.7865  
829500/1000000, happiness=0.7869 i
830000/1000000, happiness=0.7873 n
830500/1000000, happiness=0.7878  
831000/1000000, happiness=0.7882 h
831500/1000000, happiness=0.7886 i
832000/1000000, happiness=0.7891 s
832500/1000000, happiness=0.7894 t
833000/1000000, happiness=0.7899 o
833500/1000000, happiness=0.7903 r
834000/1000000, happiness=0.7908 y
834500/1000000, happiness=0.7912 "
835000/1000000, happiness=0.7917 *
835500/1000000, happ

939000/1000000, happiness=0.8795 i
939500/1000000, happiness=0.8799 n
940000/1000000, happiness=0.8804 d
940500/1000000, happiness=0.8807 e
941000/1000000, happiness=0.8812 e
941500/1000000, happiness=0.8816 r
942000/1000000, happiness=0.8820  
942500/1000000, happiness=0.8824 g
943000/1000000, happiness=0.8828 a
943500/1000000, happiness=0.8832 m
944000/1000000, happiness=0.8836 e
944500/1000000, happiness=0.8841 s
945000/1000000, happiness=0.8844 *
945500/1000000, happiness=0.8849 *
946000/1000000, happiness=0.8853 T
946500/1000000, happiness=0.8857 h
947000/1000000, happiness=0.8861 e
947500/1000000, happiness=0.8864 n
948000/1000000, happiness=0.8869  
948500/1000000, happiness=0.8873 o
949000/1000000, happiness=0.8877 n
949500/1000000, happiness=0.8881 e
950000/1000000, happiness=0.8885  
950500/1000000, happiness=0.8889 f
951000/1000000, happiness=0.8893 o
951500/1000000, happiness=0.8898 g
952000/1000000, happiness=0.8901 g
952500/1000000, happiness=0.8906 y
953000/1000000, happ